# 01 Visualize Signals（やさしいチュートリアル版）

このノートブックは、**EMA（移動平均）** と **ATR（ボラティリティ）** を使って、
「今はどんな地合いか？」「クロスのシグナルが出たか？」を順番に理解するためのミニ教科書です。  

> ゴール：読みながら動かし、パラメータをいじって「なぜこの判定になるか」を自分で説明できるようになる。

## Glossary（用語集）

- **OHLC**: 1本のローソク足を表す4つの価格。
  - **Open（始値）**: その期間の最初の価格
  - **High（高値）**: その期間で最も高い価格
  - **Low（安値）**: その期間で最も低い価格
  - **Close（終値）**: その期間の最後の価格
- **EMA（指数移動平均）**: 直近の価格に大きな重みを置く移動平均。トレンドの変化に比較的早く反応します。
- **EMA cross（EMAクロス）**: 短期EMAが長期EMAを上抜け/下抜けするイベント。売買シグナルの候補として使われます。
- **ATR（Average True Range）**: 値動きの大きさ（ボラティリティ）を表す指標。上昇/下落の方向は示しません。
- **Regime（レジーム）**: 相場状態のラベル。このノートでは ATR がしきい値より高い状態を **Active regime** と定義します。


## 0) Parameter knobs（最初に調整するつまみ）

最初にこのセルだけ編集すれば、銘柄・期間・各種パラメータを一括で変更できます。

In [ ]:
# --- Parameter knobs: まずはここをいじる ---
SYMBOL = "1306.T"          # 例: "7203.T", "AAPL"
PERIOD = "2y"              # 取得期間（例: "6mo", "1y", "2y", "5y"）
INTERVAL = "1d"            # 足種（このノートでは日足を想定）

EMA_FAST_SPAN = 12          # 短期EMA
EMA_SLOW_SPAN = 26          # 長期EMA
ATR_PERIOD = 14             # ATR期間
THRESHOLD_WINDOW = 60       # ATRしきい値に使う移動中央値の窓
LAST_N_ROWS = 12            # 最後に表示する行数（学習用テーブル）

## 1) Data loading（データ取得）

In [ ]:
from quantlab.data import fetch_ohlc
from quantlab.indicators import ema, atr
from quantlab.plot import plot_price_ema, plot_atr_regime, plot_cross_points
import pandas as pd

# データ取得
# fetch_ohlc は通常 OHLCV を含むDataFrameを返します。
df = fetch_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL).copy()

# 指標計算（列名は後続セルでも使うため先に作成）
df["EMA_FAST"] = ema(df["Close"], EMA_FAST_SPAN)
df["EMA_SLOW"] = ema(df["Close"], EMA_SLOW_SPAN)
df["ATR"] = atr(df, ATR_PERIOD)

# EMA差分（クロス判定の基本）
df["ema_diff"] = df["EMA_FAST"] - df["EMA_SLOW"]

df.tail(3)

## 2) Data sanity check（データ健全性チェック）

分析に入る前に、次を確認します。
- 冒頭/末尾の行（head/tail）
- カラム一覧
- 日付範囲
- NaN 件数（指標計算の最初にNaNが出るのは自然）

In [ ]:
# 先頭と末尾を確認
print("=== df.head() ===")
display(df.head())
print("=== df.tail() ===")
display(df.tail())

# カラム一覧
print("\n=== columns ===")
print(df.columns.tolist())

# 日付範囲（DatetimeIndex想定。違う場合にも対応）
if isinstance(df.index, pd.DatetimeIndex):
    start_date = df.index.min()
    end_date = df.index.max()
else:
    # indexが日時でない場合は、Date列などを探すためのフォールバック
    start_date = df.index.min()
    end_date = df.index.max()
print("\n=== date range ===")
print(f"start: {start_date}")
print(f"end  : {end_date}")

# NaN 件数
print("\n=== NaN counts ===")
print(df.isna().sum())

## 3) Explain EMA and ATR with numbers（数値で理解する）

### True Range（TR）と ATR の考え方

ギャップを含めた1日の値動きを捉えるために、**True Range（TR）** を使います。

\[
TR_t = \max(H_t - L_t,\ |H_t - C_{t-1}|,\ |L_t - C_{t-1}|)
\]

- \(H_t\): 当日の高値
- \(L_t\): 当日の安値
- \(C_{t-1}\): 前日の終値

そして ATR は TR の移動平均（ここでは `N=ATR_PERIOD`）です。

\[
ATR_t = moving\ average(TR, N)
\]

### EMA の考え方

EMAは「今日の価格」と「前日のEMA」を混ぜて更新します。

\[
EMA_t = \alpha \cdot price_t + (1-\alpha) \cdot EMA_{t-1}
\]

一般に \(\alpha = \frac{2}{N+1}\)（`N` は EMA span）です。

このあと、実際のデータで **数行分の手計算に近い確認** をして、
式と実装がどうつながるかを見ます。


In [ ]:
# --- Numeric example 1: True Range と ATR を数行で確認 ---
# TRの3候補を明示して「どれがmaxになるか」を確認する学習用テーブルです。
tr_example = df[["High", "Low", "Close"]].copy()
tr_example["C_prev"] = tr_example["Close"].shift(1)
tr_example["H-L"] = tr_example["High"] - tr_example["Low"]
tr_example["|H-C_prev|"] = (tr_example["High"] - tr_example["C_prev"]).abs()
tr_example["|L-C_prev|"] = (tr_example["Low"] - tr_example["C_prev"]).abs()
tr_example["TR_manual"] = tr_example[["H-L", "|H-C_prev|", "|L-C_prev|"]].max(axis=1)
tr_example["ATR_manual"] = tr_example["TR_manual"].rolling(ATR_PERIOD).mean()

print(f"=== TR / ATR numeric example (last {LAST_N_ROWS} rows) ===")
display(
    tr_example[[
        "High", "Low", "Close", "C_prev",
        "H-L", "|H-C_prev|", "|L-C_prev|",
        "TR_manual", "ATR_manual"
    ]].tail(LAST_N_ROWS)
)

print()
print("Check ATR implementation gap (should be near 0 after warm-up):")
atr_compare = (tr_example["ATR_manual"] - df["ATR"]).dropna()
print(atr_compare.tail(3))

# --- Numeric example 2: EMA の更新式を数行で確認 ---
alpha_fast = 2 / (EMA_FAST_SPAN + 1)
ema_example = df[["Close", "EMA_FAST"]].copy()
ema_example["EMA_prev"] = ema_example["EMA_FAST"].shift(1)
ema_example["EMA_formula"] = alpha_fast * ema_example["Close"] + (1 - alpha_fast) * ema_example["EMA_prev"]
ema_example["formula_gap"] = ema_example["EMA_FAST"] - ema_example["EMA_formula"]

print()
print(f"=== EMA numeric example (alpha={alpha_fast:.4f}, last {LAST_N_ROWS} rows) ===")
display(ema_example[["Close", "EMA_prev", "EMA_formula", "EMA_FAST", "formula_gap"]].tail(LAST_N_ROWS))

# --- Numeric example 3: cross logic を数行で確認 ---
cross_example = df[["EMA_FAST", "EMA_SLOW", "ema_diff"]].copy()
cross_example["ema_diff_prev"] = cross_example["ema_diff"].shift(1)
cross_example["crossed_up"] = (cross_example["ema_diff_prev"] <= 0) & (cross_example["ema_diff"] > 0)
cross_example["crossed_down"] = (cross_example["ema_diff_prev"] >= 0) & (cross_example["ema_diff"] < 0)

print()
print(f"=== EMA cross logic example (last {LAST_N_ROWS} rows) ===")
display(cross_example.tail(LAST_N_ROWS))


## 4) Explain Active regime（アクティブ判定を理解する）

ここでは、
- `ATR` の **THRESHOLD_WINDOW日移動中央値** をしきい値にする
- `ATR > threshold` の日を `active=True` とする

というルールを使います。

In [ ]:
# ATRのしきい値（移動中央値）
df["atr_threshold"] = df["ATR"].rolling(THRESHOLD_WINDOW).median()

# Active regime の真偽
df["active"] = df["ATR"] > df["atr_threshold"]

latest_atr = df["ATR"].iloc[-1]
latest_threshold = df["atr_threshold"].iloc[-1]
latest_active = bool(df["active"].iloc[-1])

print("=== Active regime (latest) ===")
print(f"ATR{ATR_PERIOD} latest : {latest_atr:.4f}")
print(f"median{THRESHOLD_WINDOW} threshold: {latest_threshold:.4f}")
print(f"active: {latest_active}")

active_ratio = df["active"].mean() * 100
print(f"\n% of days active in dataset: {active_ratio:.2f}%")

## 5) Explain crossover signals（クロスシグナルを理解する）

クロスの基本ロジック：
- `crossed_up`: 前日まで `ema_diff <= 0` で当日 `ema_diff > 0`
- `crossed_down`: 前日まで `ema_diff >= 0` で当日 `ema_diff < 0`

このノートでは、`active=True` のときだけ最終シグナルを出します。

In [ ]:
# Boolean logic を明示
crossed_up = (df["ema_diff"].shift(1) <= 0) & (df["ema_diff"] > 0)
crossed_down = (df["ema_diff"].shift(1) >= 0) & (df["ema_diff"] < 0)

# active フィルタ込みのシグナル
#   +1: BUY候補（上抜け）
#   -1: SELL候補（下抜け）
#    0: シグナルなし
df["signal"] = 0
df.loc[crossed_up & df["active"], "signal"] = 1
df.loc[crossed_down & df["active"], "signal"] = -1

print("=== logic preview (last 5) ===")
display(
    pd.DataFrame({
        "ema_diff_prev<=0": (df["ema_diff"].shift(1) <= 0),
        "ema_diff_today>0": (df["ema_diff"] > 0),
        "crossed_up": crossed_up,
        "ema_diff_prev>=0": (df["ema_diff"].shift(1) >= 0),
        "ema_diff_today<0": (df["ema_diff"] < 0),
        "crossed_down": crossed_down,
    }).tail(5)
)

# 学習用の最終テーブル
cols = ["Close", "EMA_FAST", "EMA_SLOW", "ema_diff", "ATR", "active", "signal"]
print(f"\n=== last {LAST_N_ROWS} rows ===")
display(df[cols].tail(LAST_N_ROWS))

## 6) Plot 1: Price + EMA

このプロットは、終値と短期/長期EMAの位置関係を同時に見せます。  
**なぜ重要か**: 「上昇トレンドか、下降トレンドか」「トレンド転換の初動が出ていないか」を直感的に確認できるためです。


In [ ]:
# 既存プロット関数をそのまま利用
plot_price_ema(df)

## 7) Plot 2: ATR regime

このプロットは、ATRとそのしきい値（移動中央値）を比べて、active regime を可視化します。  
**なぜ重要か**: 似たクロスでも、相場が静かなときと活発なときでは意味合いが変わるため、シグナルの質を見分ける助けになります。


In [ ]:
plot_atr_regime(df)

## 8) Plot 3: Cross points

このプロットは、EMAの上抜け/下抜けが起きた時点を価格上に重ねて表示します。  
**なぜ重要か**: エントリー候補のタイミングを視覚的に検証でき、activeフィルタがどのシグナルを採用/除外したかを説明しやすくなるからです。


In [ ]:
plot_cross_points(df)